In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import statsmodels.api as stm
import statsmodels.stats.weightstats as ws
import sklearn.linear_model as sk_lin_mod
import math
import seaborn as sns
import os
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from sklearn.feature_selection import RFE

In [ ]:
#import sys
#!{sys.executable} -m pip install --upgrade ipykernel
#!{sys.executable} -m pip install --upgrade pandas
#!{sys.executable} -m pip install --upgrade  sickit-learn 
#!{sys.executable} -m pip install featurewiz

In [ ]:
import PyQt5 #works
from PyQt5 import QtCore, QtGui, QtWidgets #works for pyqt5
%matplotlib 

In [ ]:
os.getcwd()

In [ ]:
print (os.getcwd())
currDir = os.getcwd()
fullPath = currDir + "/CabaneASucrev0r3.csv"
type(currDir)

In [ ]:
donnee = pd.read_csv(fullPath)
stats=donnee.describe()
dimensions=donnee.shape
stats

In [ ]:
donnee.isnull().sum()

Heureusement, très peu de nulls.

Quels sont les enregistrements contenant des nulls?


In [ ]:
donnee[(donnee['Temp max.(°C)'].isna()==True) | (donnee['Précip. tot. (mm)'].isna()==True)] 

Fill na

In [ ]:
donnee.fillna(method='bfill', inplace=True)
donnee.isnull().sum()

In [ ]:
donnee.columns

Supprimer les colonnes pixel

In [ ]:
donnee.filter(like='Pixel').columns
dfSansPixel = donnee.drop(donnee.filter(like='Pixel').columns, axis=1)
dfSansPixel.columns

In [ ]:
dfSansPixel.dtypes

In [ ]:
### liste de variables (colonnes) dépendantes
list_cols_dependennt_vars = []
col_debit_seve = dfSansPixel['Débit sève (L/j)'].name
col_sucre_dans_seve = dfSansPixel['Sucre sève (%)'].name
col_pct_transmittance = dfSansPixel['Transmittance produit (%)'].name
col_productivite_seve_par_saison = dfSansPixel['Production moyenne par entaille (L)'].name
list_cols_dependennt_vars = [col_debit_seve, col_sucre_dans_seve, col_pct_transmittance, col_productivite_seve_par_saison]

In [ ]:
## convertir Classe Sirop en numérique - https://www.youtube.com/watch?v=wH_ezgftiy0&t=136s
dfSansPixel['CategClasseSirop'] = dfSansPixel['Classe Sirop'].astype('category').cat.codes
dfNumerique = dfSansPixel.select_dtypes(exclude='object').copy()
dfNumerique.dtypes

### Multiple Linear Regression


https://www.geeksforgeeks.org/ml-multiple-linear-regression-using-python/

Assumption of Regression Model : 

- Linearity: The relationship between dependent and independent variables should be linear.
- Homoscedasticity: Constant variance of the errors should be maintained.
- Multivariate normality: Multiple Regression assumes that the residuals are normally distributed.
- Lack of Multicollinearity: It is assumed that there is little or no multicollinearity in

In [ ]:
stats = dfNumerique.describe()
stats

In [ ]:
#dfNumerique[dfNumerique['CategClasseSirop']==0 | dfNumerique['Pression osmoseur (bar)']==0].count()
dfNumerique[dfNumerique['Pression osmoseur (bar)']==0].count()

In [ ]:
dfNumerique = dfNumerique.loc[dfNumerique['Pression osmoseur (bar)'] >0]

In [ ]:
corr = dfNumerique.corr()
#sns.heatmap(corr, cbar=True, cmap="Blues", center=0, annot=True, fmt=".1f")

## le résultat est une matric 22x22 pas facile à visualiser

In [ ]:
#corr['Année'].sort_values(ascending=False)
corr[(corr['Année']<1) & (corr['Année']>0.2)]['Année'].sort_values(ascending=False)

In [ ]:
sorted_pairs = corr.unstack().sort_values(kind="quicksort", ascending=False).dropna()
len(sorted_pairs)

In [ ]:
x = sorted_pairs.where((sorted_pairs <1.0) & (sorted_pairs >0) ).dropna()
list_corr = sorted_pairs.tolist()
lst = [l for l in list_corr if (l < 1) & (l> 0) & (not(math.isnan(l)))]

In [ ]:
type(corr.unstack())

In [ ]:
#sns.heatmap(corr, cbar=True, center=0, annot=True, fmt="0.2f", cmap="Blues")

### apply model and check. Then, cut out variables

In [ ]:
## seulement les variables X
dfNumerique_X_cols = dfNumerique.loc[:, ~dfNumerique.columns.isin(list_cols_dependennt_vars)].copy()

## seulement les variables Y
dfNumerique_Y_cols = dfNumerique.loc[:, dfNumerique.columns.isin(list_cols_dependennt_vars)].copy()

len(dfNumerique.columns)

#### Evaluate model

=============== Evaluating model ===========

https://www.youtube.com/watch?v=VCVhwjbI6h8

========

In [ ]:
### Fonction qui imprime le sommaire du modèle
def print_model_summary(pDfColsX, pDfColsY):
    ## Add constant
    pDfColsX = stm.add_constant(pDfColsX)
    lst_models = []
    for y in (pDfColsY.columns.values):
    
        col_y = dfNumerique_Y_cols[y]
    
        model = stm.OLS(col_y, pDfColsX).fit()
        print("=========")
        print("var depend: ", y)
    
        #lst_models.append(model)
        print(model.summary())
        #sns.distplot(model.resid, fit=sts.norm)
        #sns.histplot(model.resid, kde=True, stat="density")

In [ ]:
print("---- Sommaire dfNumerique ----")
print_model_summary(dfNumerique_X_cols, dfNumerique_Y_cols)

In [ ]:
### https://www.youtube.com/watch?v=VCVhwjbI6h8
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def analyze_model(pColsY, pDfDependentVars):
    for y in (pColsY.columns.values):    
        col_y = pColsY[y]
        X_train, X_test, y_train, y_test = \
        train_test_split(pDfDependentVars, col_y, test_size=0.25, random_state=0)


        ## transforming data    
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.fit_transform(X_test)


        ## Fitting Multiple Linear Regression to the training set
        regressor =  LinearRegression()
        regressor.fit(X_train, y_train)
        

        y_pred = regressor.predict(X_test)
        mse = math.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        #r_sq = regressor.score(X_test, y_test)

        ##adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
        ## Where n is the sample size and p is the number of independent variables.

        #print ("len(X_test): ", len(X_test))
        #print ("len(y_test): ", len(y_test))
    
        ### ajd_r2
        ## https://stackoverflow.com/questions/51038820/how-to-calculated-the-adjusted-r2-value-using-scikit
        ## https://www.dummies.com/article/business-careers-money/business/accounting/calculation-analysis/how-to-calculate-the-adjusted-coefficient-of-determination-146054/
        #print ("n: ", n)
        #print ("p: ", p)
        #print ("adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)")

        n = len(X_test)
        p = len(pDfDependentVars.columns)
        adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    
        print("--------")
        
        print("[col_y]: ", col_y.name)
        print("[mse]: ", mse)
        #print("[r2] - A value of 1 indicates that the response variable can be perfectly explained by the predictor variables.")
        print("[r2]: ", r2)
        print("[adj_r2]: ", adj_r2)
        print("[coef]: ", regressor.coef_)
        print("[intercept_]: ", regressor.intercept_)
        #print("[r_sq]: ", r_sq)

In [ ]:
analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)

\---
 At this point, we have very high accuracy (r2 and adj_r2)
 can we find a smaller set of dependent vars?
\---

In [ ]:
#-------before calc high score it says all true ----

from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LinearRegression


In [ ]:
def get_reduced_df_X(pDfx, pY, pAccuracy=0.7):
    print("------")
    print("dependent var y: ", pY.name)
    
    # new X df with fewer columns than pDfx
    reducedDf = []

    # column names to keep
    dfKeeper = None

    #no of features
    nof_list=np.arange(1,len(pDfx.columns))            
    high_score=0
    #Variable to store the optimum features
    nof=0           
    score_list =[]
    for n in range(len(nof_list)):
        X_train, X_test, y_train, y_test = train_test_split(pDfx, pY, test_size = 0.3, random_state = 0)
        model = LinearRegression()
        rfe = RFE(model,n_features_to_select= nof_list[n])
        X_train_rfe = rfe.fit_transform(X_train,y_train)
        X_test_rfe = rfe.transform(X_test)
        model.fit(X_train_rfe,y_train)
        score = model.score(X_test_rfe,y_test)
        score_list.append(score)
        
        print("n: ", n)
        print("score: ", score)
        print("high_score: ", high_score)

        # print summaries for the selection of attributes
        print("rfe.support_: ",rfe.support_)
        print("rfe.ranking_: " ,rfe.ranking_)
        
        #dfKeeper = pd.DataFrame(rfe.support_,index=pDfx.columns,columns=['Rank'])
        data= {'colName': pDfx.columns, 'keep': rfe.support_ }
        dfKeeper = pd.DataFrame(data)
        dfKeeper = dfKeeper[dfKeeper["keep"]==True]
        if(score>high_score):
            high_score = score
            nof = nof_list[n]
        if (score > pAccuracy):
            break

    
    print("Optimum number of features: %d" %nof)
    print("Score with %d features: %f" % (nof, high_score))
    print("df columns to keep: ")
    print("shape")
    print(dfNumerique_X_cols[dfKeeper["colName"]].shape)
    print(dfKeeper)
    
    reducedDf = dfNumerique_X_cols[dfKeeper["colName"]]
    
    print("reducedDf: ", reducedDf.head())
    #print("unique: ", reducedDf[dfKeeper["colName"]].unique())
    return reducedDf

In [ ]:
### clé = colonne dépendante, valeur = dataframe des colonnes avec haute influence
dict_y_colsX = {}

for y in (dfNumerique_Y_cols.columns.values):    
    col_y = dfNumerique_Y_cols[y]
    reducedDf = get_reduced_df_X(dfNumerique_X_cols, col_y)
    print("reduceDf.columns.values: ",  reducedDf.columns.values)
    print("unique values: ", reducedDf[reducedDf.columns.values[0]].unique())
    print("len: ", len(reducedDf[reducedDf.columns.values[0]]))
    print("Nan: ", reducedDf[reducedDf.columns.values[0]].isnull().sum())
    ## call to analyze_model
    #analyze_model(dfNumerique_Y_cols, reducedDf)
    #analyze_model(col_y.to_frame(), reducedDf)
    dict_y_colsX.update({col_y.name: reducedDf})

In [ ]:
### analyse le modèle de nouveau avec un dataframe réduit
#analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)
for key in (dict_y_colsX.keys()):
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]
    #print(type(dfX))
    #print(type(dfY))
    #dfX =pd.DataFrame(dict_y_colsX[key], columns=[key])
    #print("shape: ", dfX.shape)
    #print("nulls: ", dfX.isnull().sum())
    analyze_model(dfY.to_frame(), dfX)
    

In [41]:
### analyse le modèle de nouveau avec un dataframe réduit
#analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)
for key in (dict_y_colsX.keys()):
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    print_model_summary(dfX, dfY.to_frame())

key:  Production moyenne par entaille (L)
var depend:  Production moyenne par entaille (L)
                                     OLS Regression Results                                    
Dep. Variable:     Production moyenne par entaille (L)   R-squared:                       0.910
Model:                                             OLS   Adj. R-squared:                  0.909
Method:                                  Least Squares   F-statistic:                 1.602e+04
Date:                                 Thu, 24 Nov 2022   Prob (F-statistic):               0.00
Time:                                         18:39:08   Log-Likelihood:                -4029.1
No. Observations:                                 1595   AIC:                             8062.
Df Residuals:                                     1593   BIC:                             8073.
Df Model:                                            1                                         
Covariance Type:                             

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier


In [ ]:

clf = DecisionTreeClassifier()
trans = RFE(clf, n_features_to_select=5)

for y in (dfNumerique_Y_cols.columns.values):    
    col_y = dfNumerique_Y_cols[y]
    dfNumerique_X_cols_small = trans.fit_transform(dfNumerique_X_cols, col_y)
columns_retained_RFE = kepler.iloc[:, 1:].columns[trans.get_support()].values

In [ ]:

X_scaled =  StandardScaler().fit_transform(dfNumerique_X_cols) 

In [ ]:
y = dfNumerique['Transmittance produit (%)']
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled,y,test_size = 0.2,stratify=y, random_state=1)

In [ ]:
dfPenguins = sns.load_dataset("penguins")
sns.histplot(dfPenguins["flipper_length_mm"], kde=True, stat="density")

In [ ]:
#### END Evaluate model

In [ ]:
##### Vif

In [ ]:
### vif
## https://www.kdnuggets.com/2019/07/check-quality-regression-model-python.html
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [ ]:
### https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = dfNumerique_X_cols.columns


In [ ]:
len(dfNumerique_X_cols.columns)

In [ ]:
len([vif(dfNumerique_X_cols.values, i)for i in range(len(dfNumerique_X_cols.columns))])

In [ ]:

# calculating VIF for each feature

vif_data["VIF"] = [vif(dfNumerique_X_cols.values, i)
                          for i in range(len(dfNumerique_X_cols.columns))]

vif_data.sort_values(by="VIF", axis=0, kind="quicksort",  ascending=False)


In [ ]:
dfVifPlusGrand10 = vif_data[vif_data["VIF"] >10]
len(dfVifPlusGrand10)

In [ ]:
#type(dfVifPlusGrand10.columns.values.tolist())
#type(list_cols_dependennt_vars)

In [ ]:
### dfNumerique_X sans les vif >10
dfNumerique_X_cols_vif =[]
dfNumerique_X_cols_vif = dfNumerique_X_cols.loc[:, ~dfNumerique_X_cols.columns.isin(dfVifPlusGrand10["feature"].values)].copy()
#for x in (dfVifPlusGrand10["feature"].values):
#    print("col: ", x)
len(dfNumerique_X_cols_vif.columns)

In [ ]:
#type(dfNumerique_Y_cols)
dfNumerique_Y_cols.head()

In [ ]:
dfNumerique_Y_cols.columns.values[0]
dfNumerique_Y_cols['Production moyenne par entaille (L)']
#dfNumerique_Y_cols[y]

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
list_reg = []
for col in list_cols_dependennt_vars:
 #   print(col.name)
    #result = reg.fit(dfNumerique[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])], dfNumerique[col.name])
    result = reg.fit(dfNumerique.loc[:, ~dfNumerique.columns.isin([col for col in list_cols_dependennt_vars])], dfNumerique[col])
    list_reg.append(result)
    

In [ ]:
print("coefs: ", list_reg[0].coef_.round(2))
print("intercept: ", list_reg[0].intercept_)


In [ ]:
dfNumerique_X_cols_vif.columns

\#####
======= Summaries ===========
\#####


In [ ]:
col_y = dfNumerique_Y_cols[y]
model = stm.OLS(col_y, dfNumerique_X_cols).fit()
dfNumerique_Y_cols['Production moyenne par entaille (L)']
type(sts.norm)

sns.histplot(model.resid, kde=True, stat="density")

#######

##### feature selection ####

https://machinelearningmastery.com/feature-selection-for-regression-data/


In [ ]:
# example of correlation feature selection for numerical data
# compare different numbers of features selected using mutual information
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV

In [ ]:

# define dataset
X = dfNumerique_X_cols.copy()

y = dfNumerique['Débit sève (L/j)']

# define the evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)


# define the pipeline to evaluate
model = LinearRegression()
fs = SelectKBest(score_func=mutual_info_regression)
pipeline = Pipeline(steps=[('sel',fs), ('lr', model)])

# define the grid
grid = dict()
grid['sel__k'] = [i for i in range(X.shape[1]-20, X.shape[1]+1)]



In [ ]:

# define the grid search
search = GridSearchCV(pipeline, grid, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv)
# perform the search
results = search.fit(X, y)


In [ ]:

# summarize best
print('Best MAE: %.3f' % results.best_score_)
print('Best Config: %s' % results.best_params_)
# summarize all
means = results.cv_results_['mean_test_score']
params = results.cv_results_['params']
for mean, param in zip(means, params):
    print(">%.3f with: %r" % (mean, param))

In [ ]:
### https://www.youtube.com/watch?v=VCVhwjbI6h8
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(dfNumerique_X_cols_vif, dfNumerique_Y_cols, test_size=0.25, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(dfNumerique_X_cols_vif, dfNumerique_Y_cols['Production moyenne par entaille (L)'], test_size=0.25, random_state=0)

In [ ]:
## transforming data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


In [ ]:
## Fitting Multiple Linear Regression to the training set
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

regressor =  LinearRegression()
regressor.fit(X_train, y_train)


In [ ]:
y_pred = regressor.predict(X_test)
math.sqrt(mean_squared_error(y_test, y_pred))
r2_score(y_test, y_pred)

In [ ]:
for m in lst_models:
    print("==== SUMMARY ====")
    m.summary()


In [ ]:
#dfNumerique_X_cols_vif.head()

In [ ]:
duncan_prestige = stm.datasets.get_rdataset("Duncan", "carData")
Y = duncan_prestige.data['income']
X = duncan_prestige.data['education']
X = stm.add_constant(X)
print("type(Y)", type(Y))
print("type(X)", type(X))
print("len(Y): ", len(Y))
print("len(X): ", len(X))

In [ ]:
#print(Y.shape)
X.head()
#Y.head()

In [ ]:

model = stm.OLS(Y,X)
results = model.fit()
results.params

###### 

In [ ]:
## On voit qu'il y a certaines correlations parfaites
## ex: (temp moyen, temp max), (temp moyen, temp min), (moyenne entaille, episode gel/degel)
## (pression osmoseur bar, boullioire 0c), (pression osmoseur bar, quantité sirop obtenu %)
## (osmoseur heures opération, alimentation osmoseur (L/j))
## (Sucre sortie osmoseur (%), pression osmoseur)
## (alimentation osmoseur, (L/j), temps boulloire) -> 0.89
## (débit sève, heures opération/j)
## Enlevons quelques unes de ces variables et revoyons la correlation
list_col_redondantes = ['Temp min.(°C)', 'Temp max.(°C)', 'Température Bouilloire (0C)'
                       , 'Quantité de sirop obtenue (L)', 'Sucre du sirop obtenu (%)'
                       , 'Osmoseur (heures opération/j)', 'Sucre sortie osmoseur (%)'
                       , 'Temps bouilloire (h)'
                       ]

## On repète l'opération antérieur avec moins de colonnes
dfNumerique = dfNumerique.loc[:, ~dfNumerique.columns.isin(list_col_redondantes)]

corr = dfNumerique.corr()

## heatmap sans correlations parfaites ou presque parfaite:
sns.heatmap(corr, cbar=True, cmap="Blues", center=0, annot=True, fmt=".2f")

In [ ]:
### pandas.series condition https://www.youtube.com/watch?v=BgfvF6mu20c
### pour chaque var Y, imprimer les variables les plus correlées
### on applique la condition 
for col in list_cols_dependennt_vars:
    print("===========================")
    print(" y = ", col.name)
    cond_correl_plus_grand_50 = ((corr[col.name] > 0.5) & (corr[col.name] <1)) 
    print(corr[col.name][cond_correl_plus_grand_50].sort_values(ascending=False))
    
#list_cols_dependennt_vars

In [ ]:
sns.pairplot(dfNumerique)

### At this point, we have strong relationships:
- Débit sève (L/j) --> Alimentation osmoseur (L/j) [0.97]
- Sucre sève (%) --> Pression osmoseur (bar) [0.9]
- Transmittance produit (%) --> Pression osmoseur (bar)[0.81], Sucre sortie osmoseur (%) [0.68]
- Production moyenne par entaille (L) --> Nombre épisodes gel/dégel [0.95]

In [ ]:
## Transmitance
### pression_osmoseur_vs_transmittance
sns.lmplot(x='Pression osmoseur (bar)', y = 'Transmittance produit (%)', data = dfNumerique)

In [ ]:
## Le graphique ression_osmoseur_vs_transmittance montre une concentration
## dans x près de 40. Regardons s'il y a des outliers:
sns.boxplot(x='Pression osmoseur (bar)',data = dfNumerique)

In [ ]:
## effectivement, le boxplot nous montre la présence des outliers
stats = dfNumerique['Pression osmoseur (bar)'].describe()
stats

In [ ]:
sns.lmplot(x='Alimentation osmoseur (L/j)' , y='Débit sève (L/j)', data=dfNumerique)

In [ ]:
### regardons combien de 0:
sns.histplot(x='Pression osmoseur (bar)',data = dfNumerique)

In [ ]:
#sns.pairplot(dfNumerique)

In [ ]:
## correlation seulement entre les variables indépendantes

dfDependantVars= dfNumerique.loc[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])]
type(dfDependantVars)

corr2 = dfDependantVars.corr()
sns.heatmap(corr2, cbar=True, annot=True, cmap="Blues", fmt=".02f", center=0)

Multiple Linear Regression - https://www.youtube.com/watch?v=J_LnPL3Qg70


In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

## https://www.statology.org/pandas-exclude-column/
#select all columns except 'rebounds' and 'assists'
#df.loc[:, ~df.columns.isin(['rebounds', 'assists'])]
list_reg = []
for col in list_cols_dependennt_vars:
 #   print(col.name)
    #result = reg.fit(dfNumerique[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])], dfNumerique[col.name])
    result = reg.fit(dfNumerique.loc[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])], dfNumerique[col.name])
    list_reg.append(result)


#dfNumerique['Année'].name
#list_cols_dependennt_vars[0].name
#[col.name for col in list_cols_dependennt_vars]
#reg.fit(dfNumerique.loc[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])], dfNumerique['Débit sève (L/j)'])
#type(dfNumerique.loc[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])])
#dfNumeriqueIndepVar = dfNumerique[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])]

In [ ]:
print("coefs: ", list_reg[0].coef_.round(2))
print("intercept: ", list_reg[0].intercept_)

fonction vérif distr normale

In [ ]:
def print_dist_norm(pCol, pXlabel, pYlabel="Fonction de densité f(x)", pNbRuns=1000):
    grille_x = np.linspace(pCol.min(), pCol.max(), pNbRuns)
    dx=(pCol.max()-(pCol.min()))/(pNbRuns-1)
    mu, sigma = sts.norm.fit(pCol.values)
    param=sts.norm.fit(pCol.values)
    pdf = sts.norm.pdf(grille_x, mu, sigma)
    ax=pCol.plot.hist(density=True, bins = 10, color = 'blue', edgecolor = 'black')
    ax.set_xlabel(pXlabel)
    ax.plot(grille_x, pdf, linewidth=3, color = 'red')
    ax.set_ylabel(pYlabel)